# Instalación

### Instalación de Hadoop

Instalamos la versión 3.3.6 de Hadoop. Se puede probar con otras versiones más recientes como en otros cuadernos, pero siempre cuidando posteriormente que las versiones del resto de componentes (sqoop, hive, pig, etc.) sean compatibles.

In [1]:
# Descargar Hadoop 3.4.2 desde el sitio oficial de Apache
!wget https://downloads.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz

# Descomprimir el archivo tar.gz descargado
!tar -xzf hadoop-3.4.2.tar.gz

# Mover la carpeta descomprimida a /usr/local/hadoop
!mv hadoop-3.4.2/ /usr/local/hadoop

# Establecer la variable de entorno JAVA_HOME para apuntar a la instalación de Java 11
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"


# Agregar el directorio bin de Hadoop al PATH para que los comandos de Hadoop estén disponibles globalmente
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] += f":{os.environ['HADOOP_HOME']}/bin:{os.environ['HADOOP_HOME']}/sbin"

# Verificar la versión de Hadoop instalada
!hadoop version

--2026-01-18 10:47:58--  https://downloads.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1065831750 (1016M) [application/x-gzip]
Saving to: ‘hadoop-3.4.2.tar.gz’

hadoop-3.4.2.tar.gz 100%[===================>]   1016M  56.0MB/s    in 18s     

2026-01-18 10:48:17 (55.3 MB/s) - ‘hadoop-3.4.2.tar.gz’ saved [1065831750/1065831750]

Hadoop 3.4.2
Source code repository https://github.com/apache/hadoop.git -r 84e8b89ee2ebe6923691205b9e171badde7a495c
Compiled by ahmarsu on 2025-08-20T10:30Z
Compiled on platform linux-x86_64
Compiled with protoc 3.23.4
From source with checksum fa94c67d4b4be021b9e9515c9b0f7b6
This command was run using /usr/local/hadoop/share/hadoop/common/hadoop-common-3.4.2.jar


Actualizamos variables de entorno

# Flume

### Instalación de Flume



In [2]:
%%bash
wget https://dlcdn.apache.org/flume/1.11.0/apache-flume-1.11.0-bin.tar.gz
tar xzf apache-flume-1.11.0-bin.tar.gz
mv apache-flume-1.11.0-bin/ /usr/local/

--2026-01-18 10:50:28--  https://dlcdn.apache.org/flume/1.11.0/apache-flume-1.11.0-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87380462 (83M) [application/x-gzip]
Saving to: ‘apache-flume-1.11.0-bin.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 9.92M 8s
    50K .......... .......... .......... .......... ..........  0% 27.1M 6s
   100K .......... .......... .......... .......... ..........  0% 14.0M 6s
   150K .......... .......... .......... .......... ..........  0% 76.5M 5s
   200K .......... .......... .......... .......... ..........  0% 62.8M 4s
   250K .......... .......... .......... .......... ..........  0% 23.5M 4s
   300K .......... .......... .......... .......... ..........  0% 34.3M 4s
   350K .......... .......... .......... .......... ..........  0%  106M 3s


In [3]:
os.environ["FLUME_HOME"] = "/usr/local/apache-flume-1.11.0-bin"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/apache-flume-1.11.0-bin/bin"
os.environ["JAVA_OPTS"]= "-Xms400m -Xmx400m -Dcom.sun.management.jmxremote"

In [4]:
!flume-ng version

Flume 1.11.0
Source code repository: https://git.apache.org/repos/asf/flume.git
Revision: 1a15927e594fd0d05a59d804b90a9c31ec93f5e1
Compiled by rgoers on Sun Oct 16 14:44:15 MST 2022
From source with checksum bbbca682177262aac3a89defde369a37


[[1]](http://flume.apache.org/) Página Web Original

[[2]](https://data-flair.training/blogs/apache-flume-installation-tutorial/) Tutorial


### Ejecución de Flume

1. Preparación del entorno



Para ejecutar un trabajo flume tenemos que crear un archivo de configuración. Aquí he creado un fichero de configuración de ejemplo que sirve para leer datos de un fichero de texto y escribirlos en un fichero local:

In [5]:
!hdfs dfs -mkdir flume
!hdfs dfs -mkdir output

In [6]:
%%writefile flume/flume.conf
# Define the source, channel, and sink
example.sources = r1
example.channels = mem1
example.sinks = k1

# Configure the source
example.sources.r1.type = netcat
example.sources.r1.bind = localhost
example.sources.r1.port = 44444

# Configure the channel
example.channels.mem1.type = memory
example.channels.mem1.capacity = 1000
example.channels.mem1.transactionCapacity = 100

# Configure the sink
example.sinks.k1.type = logger

# Bind the source and sink to the channel
example.sources.r1.channels = mem1
example.sinks.k1.channel = mem1



Writing flume/flume.conf


3. Ejecución del Agente

In [7]:
!sudo apt install netcat

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  netcat-openbsd
The following NEW packages will be installed:
  netcat netcat-openbsd
0 upgraded, 2 newly installed, 0 to remove and 1 not upgraded.
Need to get 41.4 kB of archives.
After this operation, 126 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 netcat-openbsd amd64 1.218-4ubuntu1 [39.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 netcat all 1.218-4ubuntu1 [2,044 B]
Fetched 41.4 kB in 0s (110 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf:

In [8]:
!flume-ng agent --conf-file /content/flume/flume.conf --name example -Dflume.root.logger=INFO,console -Xmx1000m

Info: Including Hadoop libraries found via (/usr/local/hadoop/bin/hadoop) for HDFS access
Info: Including Hive libraries found via () for Hive access
+ exec /usr/lib/jvm/java-17-openjdk-amd64/bin/java -Xmx20m -Dflume.root.logger=INFO,console -Xmx1000m -cp '/usr/local/apache-flume-1.11.0-bin/lib/*:/usr/local/hadoop/etc/hadoop:/usr/local/hadoop/share/hadoop/common/lib/*:/usr/local/hadoop/share/hadoop/common/*:/usr/local/hadoop/share/hadoop/hdfs:/usr/local/hadoop/share/hadoop/hdfs/lib/*:/usr/local/hadoop/share/hadoop/hdfs/*:/usr/local/hadoop/share/hadoop/mapreduce/*:/usr/local/hadoop/share/hadoop/yarn:/usr/local/hadoop/share/hadoop/yarn/lib/*:/usr/local/hadoop/share/hadoop/yarn/*:/lib/*' -Djava.library.path=:/usr/local/hadoop/lib/native org.apache.flume.node.Application --conf-file /content/flume/flume.conf --name example
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/apache-flume-1.11.0-bin/lib/log4j-slf4j-impl-2.18.0.jar!/org/slf4j/impl/

In [9]:
!echo "hello world" | nc localhost 44444


# Sqoop

### Instalación de Sqoop

1. Download sqoop


In [10]:
%%bash
wget http://archive.apache.org/dist/sqoop/1.4.7/sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
tar -xf sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
mv sqoop-1.4.7.bin__hadoop-2.6.0 /usr/local/sqoop

--2026-01-18 10:53:41--  http://archive.apache.org/dist/sqoop/1.4.7/sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17953604 (17M) [application/x-gzip]
Saving to: ‘sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  734K 24s
    50K .......... .......... .......... .......... ..........  0% 1.43M 18s
   100K .......... .......... .......... .......... ..........  0% 24.1M 12s
   150K .......... .......... .......... .......... ..........  1% 22.5M 9s
   200K .......... .......... .......... .......... ..........  1% 1.56M 10s
   250K .......... .......... .......... .......... ..........  1% 36.7M 8s
   300K .......... .......... .......... .......... ..........  1% 25.8M 7s
   350K .......... .......... .......... 

2. Download libraries

In [11]:
%%bash
wget https://dlcdn.apache.org//commons/lang/binaries/commons-lang-2.6-bin.tar.gz
tar -xf commons-lang-2.6-bin.tar.gz
cp /content/commons-lang-2.6/commons-lang-2.6.jar /usr/local/sqoop/lib

--2026-01-18 10:54:04--  https://dlcdn.apache.org//commons/lang/binaries/commons-lang-2.6-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3418617 (3.3M) [application/x-gzip]
Saving to: ‘commons-lang-2.6-bin.tar.gz’

     0K .......... .......... .......... .......... ..........  1% 9.88M 0s
    50K .......... .......... .......... .......... ..........  2% 31.0M 0s
   100K .......... .......... .......... .......... ..........  4% 12.9M 0s
   150K .......... .......... .......... .......... ..........  5% 88.3M 0s
   200K .......... .......... .......... .......... ..........  7%  102M 0s
   250K .......... .......... .......... .......... ..........  8% 16.1M 0s
   300K .......... .......... .......... .......... .......... 10% 62.7M 0s
   350K .......... .......... .......... .......... .......... 11%  280M

3. Variables de Entorno

In [12]:
import os
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["HADOOP_COMMON_HOME"] = "/usr/local/hadoop"
os.environ["HADOOP_MAPRED_HOME"] = "/usr/local/hadoop"
os.environ["SQOOP_HOME"] = "/usr/local/sqoop"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/sqoop/bin"
os.environ["HADOOP_COMMON_LIB_NATIVE_DIR"] = "/usr/local/hadoop/lib/native"
os.environ["CLASSPATH"] = "/content/tmp"
# variables de partes no instaladas pero que sqoop busca (eliminar warnings)
os.environ["HBASE_HOME"] = "/usr/local/hadoop"
os.environ["HCAT_HOME"] ="/usr/local/hadoop"
os.environ["ACCUMULO_HOME"] ="/usr/local/hadoop"
os.environ["ZOOKEEPER_HOME"] = "/usr/local/hadoop"

In [13]:
os.environ["CLASSPATH"]

'/content/tmp'

4. Test

In [14]:
!sqoop-version

2026-01-18 10:55:42,044 INFO sqoop.Sqoop: Running Sqoop version: 1.4.7
2026-01-18 10:55:42,105 ERROR cli.SqoopParser: Could not load required method of Parser: java.lang.NoSuchMethodException: org.apache.commons.cli.Option.addValueForProcessing(java.lang.String)
	at java.base/java.lang.Class.getDeclaredMethod(Class.java:2677)
	at org.apache.sqoop.cli.SqoopParser.<clinit>(SqoopParser.java:53)
	at org.apache.sqoop.tool.SqoopTool.parseArguments(SqoopTool.java:433)
	at org.apache.sqoop.Sqoop.run(Sqoop.java:135)
	at org.apache.hadoop.util.ToolRunner.run(ToolRunner.java:82)
	at org.apache.sqoop.Sqoop.runSqoop(Sqoop.java:183)
	at org.apache.sqoop.Sqoop.runTool(Sqoop.java:234)
	at org.apache.sqoop.Sqoop.runTool(Sqoop.java:243)
	at org.apache.sqoop.Sqoop.main(Sqoop.java:252)

Sqoop 1.4.7
git commit id 2328971411f57f0cb683dfb79d19d4d19d185dd8
Compiled by maugli on Thu Dec 21 15:59:58 STD 2017


 Revisión de los drivers y librerías preinstaladas en sqoop

In [15]:
!ls "/usr/local/sqoop/lib"

ant-contrib-1.0b3.jar	       kite-data-core-1.1.0.jar
ant-eclipse-1.0-jvm1.2.jar     kite-data-hive-1.1.0.jar
avro-1.8.1.jar		       kite-data-mapreduce-1.1.0.jar
avro-mapred-1.8.1-hadoop2.jar  kite-hadoop-compatibility-1.1.0.jar
commons-codec-1.4.jar	       opencsv-2.3.jar
commons-compress-1.8.1.jar     paranamer-2.7.jar
commons-io-1.4.jar	       parquet-avro-1.6.0.jar
commons-jexl-2.1.1.jar	       parquet-column-1.6.0.jar
commons-lang-2.6.jar	       parquet-common-1.6.0.jar
commons-lang3-3.4.jar	       parquet-encoding-1.6.0.jar
commons-logging-1.1.1.jar      parquet-format-2.2.0-rc1.jar
hsqldb-1.8.0.10.jar	       parquet-generator-1.6.0.jar
jackson-annotations-2.3.1.jar  parquet-hadoop-1.6.0.jar
jackson-core-2.3.1.jar	       parquet-jackson-1.6.0.jar
jackson-core-asl-1.9.13.jar    slf4j-api-1.6.1.jar
jackson-databind-2.3.1.jar     snappy-java-1.1.1.6.jar
jackson-mapper-asl-1.9.13.jar  xz-1.5.jar


Sqlite



```
gdown --id 0BwUVtTEFk6GAMmc0WmF0SUl2d3c
mv sqlite-jdbc-3.7.2.jar "/usr/local/sqoop/lib"
```



In [16]:
%%bash
wget http://ftp.ntu.edu.tw/MySQL/Downloads/Connector-J/mysql-connector-java-5.1.49.tar.gz
tar -zxf mysql-connector-java-5.1.49.tar.gz
cd mysql-connector-java-5.1.49
mv mysql-connector-java-5.1.49-bin.jar /usr/local/sqoop/lib
ls /usr/local/sqoop/lib

ant-contrib-1.0b3.jar
ant-eclipse-1.0-jvm1.2.jar
avro-1.8.1.jar
avro-mapred-1.8.1-hadoop2.jar
commons-codec-1.4.jar
commons-compress-1.8.1.jar
commons-io-1.4.jar
commons-jexl-2.1.1.jar
commons-lang-2.6.jar
commons-lang3-3.4.jar
commons-logging-1.1.1.jar
hsqldb-1.8.0.10.jar
jackson-annotations-2.3.1.jar
jackson-core-2.3.1.jar
jackson-core-asl-1.9.13.jar
jackson-databind-2.3.1.jar
jackson-mapper-asl-1.9.13.jar
kite-data-core-1.1.0.jar
kite-data-hive-1.1.0.jar
kite-data-mapreduce-1.1.0.jar
kite-hadoop-compatibility-1.1.0.jar
mysql-connector-java-5.1.49-bin.jar
opencsv-2.3.jar
paranamer-2.7.jar
parquet-avro-1.6.0.jar
parquet-column-1.6.0.jar
parquet-common-1.6.0.jar
parquet-encoding-1.6.0.jar
parquet-format-2.2.0-rc1.jar
parquet-generator-1.6.0.jar
parquet-hadoop-1.6.0.jar
parquet-jackson-1.6.0.jar
slf4j-api-1.6.1.jar
snappy-java-1.1.1.6.jar
xz-1.5.jar


--2026-01-18 10:55:52--  http://ftp.ntu.edu.tw/MySQL/Downloads/Connector-J/mysql-connector-java-5.1.49.tar.gz
Resolving ftp.ntu.edu.tw (ftp.ntu.edu.tw)... 140.112.36.185
Connecting to ftp.ntu.edu.tw (ftp.ntu.edu.tw)|140.112.36.185|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3362563 (3.2M) [application/octet-stream]
Saving to: ‘mysql-connector-java-5.1.49.tar.gz’

     0K .......... .......... .......... .......... ..........  1%  118K 27s
    50K .......... .......... .......... .......... ..........  3%  120K 27s
   100K .......... .......... .......... .......... ..........  4%  238K 22s
   150K .......... .......... .......... .......... ..........  6% 13.8M 16s
   200K .......... .......... .......... .......... ..........  7%  239K 15s
   250K .......... .......... .......... .......... ..........  9%  239K 15s
   300K .......... .......... .......... .......... .......... 10% 29.0M 12s
   350K .......... .......... .......... .......... .......... 12%

### Ejecución de Sqoop

La base de datos Rfam es una colección de familias de secuencias de ARN estructurales que incluyen genes de ARN no codificantes, así como elementos cis-reguladores. Cada familia está representada por un alineamiento múltiple de secuencias y un modelo de covarianza (MC).

[Enlace](https://docs.rfam.org/en/latest/about-rfam.html)



```
sqoop import
  --connect jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam
  --username rfamro --password ""
  --table family --m 1
```



In [18]:
%%bash
sqoop import -Dmapreduce.job.user.classpath.first=true \
  --connect jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam  \
  --username rfamro --password "" \
  --table family \
  --m 1 \
  --bindir /usr/local/hadoop/etc/hadoop \
  --class-name family \

2026-01-18 10:56:15,836 INFO sqoop.Sqoop: Running Sqoop version: 1.4.7
2026-01-18 10:56:15,937 ERROR cli.SqoopParser: Could not load required method of Parser: java.lang.NoSuchMethodException: org.apache.commons.cli.Option.addValueForProcessing(java.lang.String)
	at java.base/java.lang.Class.getDeclaredMethod(Class.java:2677)
	at org.apache.sqoop.cli.SqoopParser.<clinit>(SqoopParser.java:53)
	at org.apache.sqoop.tool.SqoopTool.parseArguments(SqoopTool.java:433)
	at org.apache.sqoop.Sqoop.run(Sqoop.java:135)
	at org.apache.hadoop.util.ToolRunner.run(ToolRunner.java:82)
	at org.apache.sqoop.Sqoop.runSqoop(Sqoop.java:183)
	at org.apache.sqoop.Sqoop.runTool(Sqoop.java:234)
	at org.apache.sqoop.Sqoop.runTool(Sqoop.java:243)
	at org.apache.sqoop.Sqoop.main(Sqoop.java:252)

Missing argument for option: connect


CalledProcessError: Command 'b'sqoop import -Dmapreduce.job.user.classpath.first=true \\\n  --connect jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam  \\\n  --username rfamro --password "" \\\n  --table family \\\n  --m 1 \\\n  --bindir /usr/local/hadoop/etc/hadoop \\\n  --class-name family \\\n'' returned non-zero exit status 1.

In [ ]:
!hdfs dfs -cat family/* | head

2024-01-12 09:35:50,310 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
RF00001,5S_rRNA,1302,5S ribosomal RNA,Griffiths-Jones SR, Mifsud W, Gardner PP,Szymanski et al, 5S ribosomal database, PMID:11752286,38.0,38.0,37.9,5S ribosomal RNA (5S rRNA) is a component of the large ribosomal subunit in both prokaryotes and eukaryotes. In eukaryotes, it is synthesised by RNA polymerase III (the other eukaryotic rRNAs are cleaved from a 45S precursor synthesised by RNA polymerase I). In Xenopus oocytes, it has been shown that fingers 4-7 of the nine-zinc finger transcription factor TFIIIA can bind to the central region of 5S RNA. Thus, in addition to positively regulating 5S rRNA transcription, TFIIIA also stabilises 5S rRNA until it is required for transcription.,null,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 24.99 -Z 549862.597050 CM SEQDB,712,139932,0,0,Gene; rRNA;,Publish